# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

# Find the ground state using direct minimisation (always using SCF is boring ...)
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.121366e+02     1.824263e+00
 * time: 0.08679986000061035
     1     1.035474e+01     9.940937e-01
 * time: 0.8810899257659912
     2    -1.167430e+01     1.204369e+00
 * time: 1.0302529335021973
     3    -3.393064e+01     9.062136e-01
 * time: 1.1741318702697754
     4    -4.737324e+01     6.081873e-01
 * time: 1.3258209228515625
     5    -5.682074e+01     2.251743e-01
 * time: 1.472717046737671
     6    -5.956193e+01     1.962917e-01
 * time: 1.5769720077514648
     7    -6.070078e+01     8.401254e-02
 * time: 1.6883199214935303
     8    -6.123906e+01     4.150185e-02
 * time: 1.7934949398040771
     9    -6.158700e+01     3.987510e-02
 * time: 1.9045429229736328
    10    -6.179760e+01     3.269458e-02
 * time: 2.009377956390381
    11    -6.192793e+01     2.926917e-02
 * time: 2.121914863586426
    12    -6.202316e+01     2.027617e-02
 * time: 2.2272849082946777
    13    -6.209362e+01     1.526693e-02
 * time: 2.339168071746

In [4]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671072
    AtomicLocal         -18.8557675
    AtomicNonlocal      14.8522638
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485373 
    Xc                  -19.3336821

    total               -62.261666461390
